In [ ]:
!pip install sqlalchemy pandas pymysql


In [ ]:
from sqlalchemy import create_engine

engine = create_engine(".env data")


In [ ]:
import pandas as pd

def rentals_month(engine, month, year):
    # SQL query to get rental data for the given month and year
    query = f"""
    SELECT r.customer_id, r.rental_date
    FROM rental r
    WHERE MONTH(r.rental_date) = {month} AND YEAR(r.rental_date) = {year}
    """
    # Execute the query and return the result as a Pandas DataFrame
    return pd.read_sql(query, engine)


In [ ]:
def rental_count_month(rentals_df, month, year):
    # Group by customer_id and count the number of rentals
    rentals_count = rentals_df.groupby('customer_id').size().reset_index(name='rental_count')
    
    # Add the new column for the specific month and year
    rentals_count[f'rentals_{month:02d}_{year}'] = rentals_count['rental_count']
    
    # Drop the 'rental_count' column, as it's no longer needed
    rentals_count.drop(columns=['rental_count'], inplace=True)
    
    return rentals_count


In [ ]:
def compare_rentals(df1, df2):
    # Merge the two DataFrames on customer_id
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_month1', '_month2'))
    
    # Calculate the difference between the two months
    merged_df['difference'] = merged_df[f'rentals_{df1.columns[1][:2]}_{df1.columns[1][3:]}'] - merged_df[f'rentals_{df2.columns[1][:2]}_{df2.columns[1][3:]}']
    
    return merged_df


In [ ]:
rentals_may = rentals_month(engine, 5, 2005)
rentals_june = rentals_month(engine, 6, 2005)

# Get the rental counts for both months
rentals_count_may = rental_count_month(rentals_may, 5, 2005)
rentals_count_june = rental_count_month(rentals_june, 6, 2005)

# Compare rentals between May and June
comparison_df = compare_rentals(rentals_count_may, rentals_count_june)

# Display the comparison DataFrame
print(comparison_df)